In [ ]:
# Essential imports for image colorization
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage import data, color, filters, segmentation, measure
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
plt.style.use('seaborn-v0_8')

print("🎨 Image Colorization Laboratory Ready!")
print("🌈 Bringing color back to black and white memories")
print(f"OpenCV version: {cv2.__version__}")
print(f"TensorFlow version: {tf.__version__}")

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Load and explore LAB color space
def explore_lab_colorspace():
    """Comprehensive exploration of LAB color space"""
    
    # Load sample color images
    images = {
        'Astronaut': data.astronaut(),
        'Coffee': data.coffee(),
        'Coins': data.coins(),
        'Flowers': data.chelsea()
    }
    
    # Convert to LAB and analyze
    lab_analysis = {}
    
    for name, img in images.items():
        # Convert to LAB
        lab_img = color.rgb2lab(img)
        
        # Extract channels
        L_channel = lab_img[:, :, 0]
        a_channel = lab_img[:, :, 1]
        b_channel = lab_img[:, :, 2]
        
        # Calculate statistics
        analysis = {
            'L_range': (np.min(L_channel), np.max(L_channel)),
            'a_range': (np.min(a_channel), np.max(a_channel)),
            'b_range': (np.min(b_channel), np.max(b_channel)),
            'L_mean': np.mean(L_channel),
            'a_mean': np.mean(a_channel),
            'b_mean': np.mean(b_channel),
            'L_std': np.std(L_channel),
            'a_std': np.std(a_channel),
            'b_std': np.std(b_channel)
        }
        
        lab_analysis[name] = {
            'original': img,
            'lab': lab_img,
            'L': L_channel,
            'a': a_channel,
            'b': b_channel,
            'stats': analysis
        }
    
    return lab_analysis

def visualize_lab_channels(lab_data, image_name):
    """Visualize LAB channels for a specific image"""
    
    data_item = lab_data[image_name]
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle(f'LAB Color Space Analysis: {image_name}', fontsize=16, fontweight='bold')
    
    # Original image
    axes[0, 0].imshow(data_item['original'])
    axes[0, 0].set_title('Original RGB')
    axes[0, 0].axis('off')
    
    # L channel
    axes[0, 1].imshow(data_item['L'], cmap='gray')
    axes[0, 1].set_title('L Channel (Lightness)')
    axes[0, 1].axis('off')
    
    # a channel
    axes[0, 2].imshow(data_item['a'], cmap='RdGy_r')
    axes[0, 2].set_title('a Channel (Green-Red)')
    axes[0, 2].axis('off')
    
    # b channel
    axes[0, 3].imshow(data_item['b'], cmap='YlBu_r')
    axes[0, 3].set_title('b Channel (Blue-Yellow)')
    axes[0, 3].axis('off')
    
    # Channel histograms
    channels = ['L', 'a', 'b']
    colors = ['gray', 'red', 'blue']
    
    for i, (channel, color_hist) in enumerate(zip(channels, colors)):
        channel_data = data_item[channel].flatten()
        axes[1, i].hist(channel_data, bins=50, alpha=0.7, color=color_hist)
        axes[1, i].set_title(f'{channel} Channel Histogram')
        axes[1, i].set_xlabel('Value')
        axes[1, i].set_ylabel('Frequency')
        axes[1, i].grid(True, alpha=0.3)
    
    # Statistics
    stats = data_item['stats']
    stats_text = f"""L: {stats['L_range'][0]:.1f} to {stats['L_range'][1]:.1f} (μ={stats['L_mean']:.1f}, σ={stats['L_std']:.1f})
a: {stats['a_range'][0]:.1f} to {stats['a_range'][1]:.1f} (μ={stats['a_mean']:.1f}, σ={stats['a_std']:.1f})
b: {stats['b_range'][0]:.1f} to {stats['b_range'][1]:.1f} (μ={stats['b_mean']:.1f}, σ={stats['b_std']:.1f})"""
    
    axes[1, 3].text(0.1, 0.5, stats_text, transform=axes[1, 3].transAxes, 
                    fontsize=11, verticalalignment='center')
    axes[1, 3].set_title('Channel Statistics')
    axes[1, 3].axis('off')
    
    plt.tight_layout()
    plt.show()

# Explore LAB color space
print("🔍 Exploring LAB Color Space...")
lab_data = explore_lab_colorspace()

# Visualize for multiple images
for image_name in ['Astronaut', 'Coffee']:
    visualize_lab_channels(lab_data, image_name)

# Print comprehensive analysis
print("\\n📊 LAB Color Space Analysis Summary:")
print("=" * 90)
print(f"{'Image':<12} | {'L Range':<15} | {'a Range':<15} | {'b Range':<15} | {'Dominant':<12}")
print("-" * 90)

for name, data_item in lab_data.items():
    stats = data_item['stats']
    L_range = f"{stats['L_range'][0]:.1f}-{stats['L_range'][1]:.1f}"
    a_range = f"{stats['a_range'][0]:.1f}-{stats['a_range'][1]:.1f}"
    b_range = f"{stats['b_range'][0]:.1f}-{stats['b_range'][1]:.1f}"
    
    # Determine dominant color characteristics
    if abs(stats['a_mean']) > abs(stats['b_mean']):
        dominant = "Red-Green" if stats['a_mean'] > 0 else "Green-Red"
    else:
        dominant = "Yellow" if stats['b_mean'] > 0 else "Blue"
    
    print(f"{name:<12} | {L_range:<15} | {a_range:<15} | {b_range:<15} | {dominant:<12}")

print("\\n💡 Key Insights:")
print("• L channel preserves all structural information")
print("• a,b channels encode color information independently")
print("• Different images have characteristic color signatures")
print("• LAB space enables precise color manipulation")
